## Email

In [2]:
import os 
import pandas as pd
import json
import ast

# read four dirs email.csv
# weeks = range(1, 5)
weeks = range(1, 4)
week_dates = [5, 5, 5, 5] # IMPORTANT!
week_pointer = 0

foundation_model = "openai"

scenario = "Tech-Company"  # 1: change
company_type = "tech" # 2: change
# company_type = "tech_company" # 3: change
company_type_member = "tech_company" # 4: change

# scenario = "Finance-Company"
# company_type = "finance"
# company_type_member = "finance_corporation"

# scenario = "Medical-Institution"
# company_type_member = "medical_institution"
# company_type = "medical"

# Reading
log_root_dir = f'/data2/visitor/ASE25/Chimera-Dataset/'
root_dir = "/data2/visitor/ASE25/Chimera/"

# concat all the email.csv
email_csvs = pd.DataFrame()

for week in weeks:
    weekly_dir = os.path.join(log_root_dir, scenario, f'week{week}-{foundation_model}-{company_type}')
    weekly_email_log_path = os.path.join(weekly_dir, "execution_logs", 'email.csv')
    
    df = pd.read_csv(weekly_email_log_path, encoding='utf-8')

    # Concatenate all DataFrames
    email_csvs = pd.concat([email_csvs, df], ignore_index=True)

# update a new column "sim_timestamp" with date and the time. The date start from starting_date and add 1 day when the next cell(time) is smaller than the previous cell
starting_date = "2025-05-01"
# email_csvs['sim_timestamp'] = pd.to_datetime(starting_date) + pd.to_timedelta(email_csvs.index, unit='d')

#### settle the sim timestamp with proper date ####
date_counter = 1
for i in range(len(email_csvs)):
    sim_time = pd.to_datetime(email_csvs.loc[i, 'sim_timestamp'], format='%H:%M:%S')
    
    if i == 0:
        sim_date = starting_date
    else:
        prev_time = pd.to_datetime(email_csvs.loc[i-1, 'sim_timestamp'], format='%H:%M:%S')
        if (sim_time < prev_time) and ((prev_time - sim_time) > pd.Timedelta(hours=6)):
            if date_counter == week_dates[week_pointer]:
                # print("next week")
                add_date = 8 - week_dates[week_pointer]
                week_pointer += 1
                date_counter = 1
            else:
                date_counter += 1
                add_date = 1
            sim_date = (pd.to_datetime(sim_date) + pd.Timedelta(days=add_date)).strftime('%Y-%m-%d')
            # print(f"Date changed to {sim_date} at index {i}")
    email_csvs.loc[i, 'full_sim_timestamp'] = f"{sim_date} {email_csvs.loc[i, 'sim_timestamp']}"

In [3]:
email_csvs.head(5)

,email_from,real_timestamp,sim_timestamp,name,email_to,email_cc,subject,content,full_sim_timestamp
0,tart-1,2025-05-16 15:15:27,08:52:21,Ayana Nakamura,3dmod-1,['3dmod-2'],Discussion on Asset Formats and Capabilities,"Hi Team,\n\nI hope this message finds you well...",2025-05-01 08:52:21
1,ndes-1,2025-05-16 15:15:32,08:54:51,Chloe Dubois,ldes-1,['sdes-1'],Brainstorming Narrative Themes and Game Mechanics,"Hi Team,\n\nI hope this message finds you well...",2025-05-01 08:54:51
2,cdev-2,2025-05-16 15:15:36,08:55:31,Javier Rodriguez,ldes-1,"['uiux-1', 'ndes-1']",Key User Interaction Features Brainstorming Se...,"Hi Team,\n\nAs we continue to develop our thir...",2025-05-01 08:55:31
3,3dmod-2,2025-05-16 15:15:40,08:57:33,Ricardo Oliveira,tart-1,[],Re: Discussion on Asset Formats and Capabilities,"Hi Ayana,\n\nThank you for your email and for ...",2025-05-01 08:57:33
4,prod-1,2025-05-16 15:15:40,08:55:04,Olivia Rodriguez,lpro-1,"['sdes-1', 'sdes-2']",Key Milestones Discussion for Our Project,"Hi Team,\n\nI hope this email finds you well! ...",2025-05-01 08:55:04


In [4]:
# Add the missing columns

### 1. add id list with content from email_from insert next to the index
email_csvs.insert(0, 'id', email_csvs['email_from'])

id_role_map = {}
id_list = []
profile_list = []

profile_output_dir = f"{root_dir}/experiment_output/{foundation_model}_{company_type_member}/generated_members"
for file in os.listdir(profile_output_dir):
    if file.endswith(".jsonc"):
        member_profile_path = os.path.join(profile_output_dir, file)
        with open(member_profile_path, 'r') as f:
            member_profile = json.load(f)
        id_role_map[member_profile['id']] = member_profile['role'] # add id-role map
        id_list.append(member_profile['id'])
        profile_list.append(member_profile) # add profile

id_email_map = {profile['id']: profile['email'] for profile in profile_list}
id_pc_map = {profile['id']: profile['container_id'] for profile in profile_list}

### 2. replace email_from with the email
email_csvs['email_from'] = email_csvs['email_from'].map(id_email_map)

### 3. replace email_to with the email
email_csvs['email_to'] = email_csvs['email_to'].map(id_email_map)
# change the column name to "to"
email_csvs.rename(columns={'email_to': 'to'}, inplace=True)

### 4. replace email_cc list
for i in range(len(email_csvs)):
    if email_csvs.loc[i, 'email_cc'] == '[]':
        email_cc_out = "NaN"
    else:
        email_cc_list = ast.literal_eval(email_csvs.loc[i, 'email_cc'])
        email_cc_out_list = []
        for email_des in email_cc_list:
            # go through map id_email_map to find the email
            # to lower case
            email_des = email_des.lower()
            email_cc_out_list.append(id_email_map.get(email_des, email_des))
            email_cc_out = ', '.join(email_cc_out_list)
    email_csvs.loc[i, 'email_cc'] = email_cc_out

### 5. add new column named "size" to calculate the word count of the email_content
content_idx = email_csvs.columns.get_loc('content')
email_csvs.insert(content_idx+1, 'size', email_csvs['content'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0))

### 6. add new column named "attachment" with value of "0"
email_csvs.insert(email_csvs.columns.get_loc('size') + 1, 'attachments', 0)

### 7. add new column named "pc" with profile content of "container_id"
email_csvs.insert(email_csvs.columns.get_loc('name') + 1, 'pc', email_csvs['id'].map(id_pc_map))

### 8. add new column named "bcc"
email_csvs.insert(email_csvs.columns.get_loc('email_cc') + 1, 'bcc', 'NaN')

In [5]:
email_csvs.head(5)

,id,email_from,real_timestamp,sim_timestamp,name,pc,to,email_cc,bcc,subject,content,size,attachments,full_sim_timestamp
0,tart-1,tart-451284@corp.com,2025-05-16 15:15:27,08:52:21,Ayana Nakamura,a11de2a1e450,mmdl-564321@corp.com,mmdl-123456@corp.com,NaN,Discussion on Asset Formats and Capabilities,"Hi Team,\n\nI hope this message finds you well...",132,0,2025-05-01 08:52:21
1,ndes-1,narr-394856@corp.com,2025-05-16 15:15:32,08:54:51,Chloe Dubois,f420e78c9c8d,ldes-981247@corp.com,sdes-294567@corp.com,NaN,Brainstorming Narrative Themes and Game Mechanics,"Hi Team,\n\nI hope this message finds you well...",212,0,2025-05-01 08:54:51
2,cdev-2,cdev-782419@corp.com,2025-05-16 15:15:36,08:55:31,Javier Rodriguez,289d67b61f3c,ldes-981247@corp.com,"udsg-873294@corp.com, narr-394856@corp.com",NaN,Key User Interaction Features Brainstorming Se...,"Hi Team,\n\nAs we continue to develop our thir...",157,0,2025-05-01 08:55:31
3,3dmod-2,mmdl-123456@corp.com,2025-05-16 15:15:40,08:57:33,Ricardo Oliveira,f39f8d2e7b34,tart-451284@corp.com,NaN,NaN,Re: Discussion on Asset Formats and Capabilities,"Hi Ayana,\n\nThank you for your email and for ...",266,0,2025-05-01 08:57:33
4,prod-1,prod-395621@corp.com,2025-05-16 15:15:40,08:55:04,Olivia Rodriguez,a4c6d7e8b123,prog-578832@corp.com,"sdes-294567@corp.com, sdes-294568@corp.com",NaN,Key Milestones Discussion for Our Project,"Hi Team,\n\nI hope this email finds you well! ...",180,0,2025-05-01 08:55:04


In [6]:
# output_dir = f"{root_dir}/Final-Output/{company_type_member}/"
output_dir = f"{root_dir}/Final-Output-OpenAI/{company_type_member}/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

sim_idx = email_csvs.columns.get_loc("sim_timestamp")
full_sim = email_csvs.pop("full_sim_timestamp")
email_csvs.insert(sim_idx + 1, "full_sim_timestamp", full_sim)

# rename the column "name" to "user"
email_csvs.rename(columns={'name': 'user'}, inplace=True)
# rename the column "full_sim_timestamp" to "time"
email_csvs.rename(columns={'full_sim_timestamp': 'date'}, inplace=True)
# rename the column "email_cc" to "cc"
email_csvs.rename(columns={'email_cc': 'cc'}, inplace=True)
email_csvs.rename(columns={'email_from': 'from'}, inplace=True)

# remove the column "real_timestamp" and "sim_timestamp"
email_csvs.drop(columns=['real_timestamp', 'sim_timestamp'], inplace=True)

# sorting by time
email_csvs = email_csvs.sort_values(by='date', ascending=True).reset_index(drop=True)

# save the email_csvs to csv
email_csvs.to_csv(f'{output_dir}/{company_type_member}_email.csv', index=False, encoding='utf-8')

## Remove duplicate column (Not Necessary)

In [14]:
# remove email index
import pandas as pd
csv_file = "/data2/visitor/ASE25/Chimera/Final-Output/tech_company/tech_email.csv"
email_df = pd.read_csv(csv_file)
email_df = email_df.drop(email_df.columns[0], axis=1)
email_df.to_csv(csv_file, index=False, encoding='utf-8')